In [ ]:
# pip install -q sentence-transformers
# pip install pymupdf
# pip install marker-pdf

In [12]:
import os

docs = 'docs/'
files = [f for f in os.listdir(docs) if os.path.isfile(os.path.join(docs, f))]
print(files)

['Lärarnas frånvaro under coronapandemin.pdf', 'Äldres deltagande på arbetsmarknaden 2001–2023 Arbetskraftsundersökningarna.pdf', 'Det yrkesmässiga fisket i havet, oktober 2023.pdf', 'Sveriges framtida befolkning 2024–2070.pdf', 'Det yrkesmässiga fisket i havet, december 2023.pdf', 'Fritidsfiske 2022.pdf']


In [48]:
import importlib
import marker.convert
importlib.reload(marker.convert)

from marker.models import load_all_models


import convert_pdf_to_csv
importlib.reload(convert_pdf_to_csv)
import convert_pdf_to_csv

model_lst = load_all_models()
all_pages, full_text, images, out_meta = convert_pdf_to_csv.convert_single_pdf(docs+files[3], model_lst)

#full_text, images, out_meta = convert_single_pdf(docs+files[2], model_lst, max_pages=5)

/home/buddy/SCB-RAG/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded detection model vikp/surya_det2 on device cuda with dtype torch.float16
Loaded detection model vikp/surya_layout2 on device cuda with dtype torch.float16
Loaded reading order model vikp/surya_order on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec on device cuda with dtype torch.float16
Loaded texify model to cuda with torch.float16 dtype


Finding reading order: 100%|██████████| 42/42 [00:11<00:00,  3.63it/s]


251
[MergedBlock(bbox=[221.2, 52.9, 333.2, 80.9], lines=[MergedLine(bbox=[221.2, 52.9, 239.3, 60.9], text='2024', fonts=['roboto-regular_non_symbolic_use_extern_attr', 'none_']), MergedLine(bbox=[221.2, 62.8, 333.2, 70.9], text='Demografiska rapporter 2024:1', fonts=['roboto-regular_non_symbolic_use_extern_attr', 'none_']), MergedLine(bbox=[221.2, 72.9, 323.6, 80.9], text='Demographic reports 2024:1', fonts=['roboto-regular_non_symbolic_use_extern_attr', 'none_', 'roboto-regular_non_symbolic_use_extern_attr', 'none_'])], pnum=1, block_type='Text'), MergedBlock(bbox=[221.2, 166.7, 475.8, 214.7], lines=[MergedLine(bbox=[221.2, 166.7, 429.1, 190.7], text='Sveriges framtida ', fonts=['ptserif-bold_non_symbolic_use_extern_attr', 'none_']), MergedLine(bbox=[221.2, 190.7, 475.8, 214.7], text='befolkning 2024–2070', fonts=['ptserif-bold_non_symbolic_use_extern_attr', 'bcdgee+ptserif-bold_non_symbolic_use_extern_attr', 'ptserif-bold_non_symbolic_use_extern_attr', 'none_'])], pnum=1, block_type=

In [52]:
def split_into_chunks(pages):
    chunks = []
    current_chunk = []

    for page_num, page in enumerate(pages, start=1):
        lines = page.split('\n')

        for line in lines:
            if line.startswith('|') and current_chunk and current_chunk[-1].startswith('|'):
                current_chunk.append(line)
            elif line.strip() == '' and current_chunk:
                joined_sentence_chunk = '\n'.join(current_chunk)
                chunk_dict = {
                    'page_num': page_num,
                    'chunk_text': joined_sentence_chunk,
                    'chunk_char_count': len(joined_sentence_chunk),
                    'chunk_word_count': len([word for word in joined_sentence_chunk.split(" ")]),
                    'chunk_token_count': len(joined_sentence_chunk) / 4
                }
                chunks.append(chunk_dict)
                current_chunk = []
            else:
                current_chunk.append(line)

        if current_chunk:
            joined_sentence_chunk = '\n'.join(current_chunk)
            chunk_dict = {
                'page_num': page_num,
                'chunk_text': joined_sentence_chunk,
                'chunk_char_count': len(joined_sentence_chunk),
                'chunk_word_count': len([word for word in joined_sentence_chunk.split(" ")]),
                'chunk_token_count': len(joined_sentence_chunk) / 4
            }
            chunks.append(chunk_dict)
            current_chunk = []

    return chunks

# Assuming the pages are stored in a list called 'pages'

chunks = split_into_chunks(all_pages)

for i, chunk_dict in enumerate(chunks, start=1):
    print(f"Chunk {i} (Page {chunk_dict['page_num']}):")
    print(f"Text: {chunk_dict['chunk_text']}")
    print(f"Character Count: {chunk_dict['chunk_char_count']}")
    print(f"Word Count: {chunk_dict['chunk_word_count']}")
    print(f"Token Count: {chunk_dict['chunk_token_count']}")
    print()

Chunk 1 (Page 1):
Text: Sveriges framtida befolkning 2024–2070
Character Count: 38
Word Count: 4
Token Count: 9.5

Chunk 2 (Page 2):
Text: 2024 Demografiska rapporter 2024:1 Demographic reports 2024:1
Character Count: 61
Word Count: 7
Token Count: 15.25

Chunk 3 (Page 2):
Text: ## Sveriges Framtida Befolkning 2024–2070
Character Count: 41
Word Count: 5
Token Count: 10.25

Chunk 4 (Page 2):
Text: | Producent     | SCB, Statistiska centralbyrån Avdelningen för social statistik och  analys 171 54 Solna +46 10-479 50 00   |
|---------------|------------------------------------------------------------------------------------------------------------|
| Förfrågningar | Lena Lundkvist +46 10-479 46 78 lena.lundkvist@scb.se eller  demografi@scb.se                              |
Character Count: 380
Word Count: 69
Token Count: 95.0

Chunk 5 (Page 2):
Text: Du får kopiera och på annat sätt mångfaldiga innehållet. Vi vill dock att du uppger källa på följande sätt: Källa: SCB, Sveriges framtida bef

In [65]:
from tqdm.auto import tqdm # for progress bars, requires !pip install tqdm 

def pages_to_dict(pages):
  page_list = []
  for page_number, page in enumerate(pages):
    text = page.replace("\n", " ").strip() 
    page_list.append({
      "page_number": page_number+1,
      "page_char_count": len(text),
      "page_word_count": len(text.split(" ")),
      "page_sentence_count_raw": len(text.split(". ")),
      "page_token_count": len(text) / 4,
      "text": text
    })
  return page_list

page_dicts = pages_to_dict(all_pages)

In [70]:
from spacy.lang.en import English
import pandas as pd
import re


nlp = English()
nlp.add_pipe("sentencizer")
for item in tqdm(page_dicts):
    item["sentences"] = list(nlp(item["text"]).sents)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    item["sentence_count"] = len(item["sentences"])

num_sentence_chunk_size = 10 

def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(page_dicts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

pages_and_chunks = []
for item in tqdm(page_dicts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4
        
        pages_and_chunks.append(chunk_dict)

df = pd.DataFrame(pages_and_chunks)
min_token_length = 30
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

100%|██████████| 251/251 [00:00<00:00, 486268.04it/s]


In [74]:
import re

pages_and_chunks = []
for item in tqdm(page_dicts):
  for sentence_chunk in item["sentence_chunks"]:
    chunk_dict = {}
    chunk_dict["page_number"] = item["page_number"]
    
    joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
    joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
    chunk_dict["sentence_chunk"] = joined_sentence_chunk

    chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
    chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
    chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
    
    pages_and_chunks.append(chunk_dict)



df = pd.DataFrame(pages_and_chunks)
min_token_length = 30
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

100%|██████████| 251/251 [00:00<00:00, 14049.11it/s]


413

In [112]:
from sentence_transformers import SentenceTransformer, util
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device="cuda") 

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32, 
                                               convert_to_tensor=True) 

for i in range(len(pages_and_chunks_over_min_token_len)):
    pages_and_chunks_over_min_token_len[i]["embedding"] = text_chunk_embeddings[i].tolist()

column_to_remove = "embeddings"  # replace with the name of the column you want to remove

for item in pages_and_chunks_over_min_token_len:
    if column_to_remove in item:
        del item[column_to_remove]

/home/buddy/SCB-RAG/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [113]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

# Semantic Seach

In [4]:
import pandas as pd
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,2,2024 Demografiska rapporter 2024:1 Demographic...,1558,192,389.50,"[0.04175179451704025, 0.07661864906549454, -0...."
1,3,## Förord I denna rapport redovisas en framskr...,1093,149,273.25,"[0.048895955085754395, -0.028946656733751297, ..."
2,3,Till denna rapport har flera personer bidragit...,447,62,111.75,"[0.06070384010672569, 0.041797105222940445, -0..."
3,4,## Innehåll | Förord 3 Sammanfattning 9 Osäker...,1392,102,348.00,"[0.023769743740558624, -0.01564064249396324, -..."
4,5,| Antaganden för födda i Norden eller EU utom ...,1657,123,414.25,"[0.03763386607170105, 0.05547052621841431, -0...."


In [5]:
import ast
import torch
import numpy as np 
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.array(ast.literal_eval(x)))
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([401, 768])

In [116]:
text_chunks_and_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,2,2024 Demografiska rapporter 2024:1 Demographic...,1558,192,389.50,"[0.04175179451704025, 0.07661864906549454, -0...."
1,3,## Förord I denna rapport redovisas en framskr...,1093,149,273.25,"[0.048895955085754395, -0.028946656733751297, ..."
2,3,Till denna rapport har flera personer bidragit...,447,62,111.75,"[0.06070384010672569, 0.041797105222940445, -0..."
3,4,## Innehåll | Förord 3 Sammanfattning 9 Osäker...,1392,102,348.00,"[0.023769743740558624, -0.01564064249396324, -..."
4,5,| Antaganden för födda i Norden eller EU utom ...,1657,123,414.25,"[0.03763386607170105, 0.05547052621841431, -0...."


In [43]:
import torch
from time import perf_counter as timer
from sentence_transformers import SentenceTransformer, util
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device=device)

queries = ["Vilka åldersgrupper kommer att vara mest påverkade av förändringar i dödligheten?",
           "Hur många barn i grundskoleåldern kommer att finnas i Sverige år 2070?"]

query = queries[0]
print(f"Query: {query}")

query_embedding = embedding_model.encode(query, convert_to_tensor=True)

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

top_results_dot_product = torch.topk(dot_scores, k=5)
top_chunks_dot_product = [{'page_number': pages_and_chunks[i]['page_number'], 'sentence_chunk': pages_and_chunks[i]['sentence_chunk']} for i in top_results_dot_product.indices.tolist()]
top_results_dot_product

Query: Vilka åldersgrupper kommer att vara mest påverkade av förändringar i dödligheten?
Time take to get scores on 401 embeddings: 0.00006 seconds.


torch.return_types.topk(
values=tensor([0.7185, 0.6966, 0.6894, 0.6834, 0.6654], device='cuda:0'),
indices=tensor([ 53,  28,  43, 161,  59], device='cuda:0'))

In [44]:
top_chunks_dot_product

[{'page_number': 29,
  'sentence_chunk': 'De kommande tio åren förväntas gruppen att minska något, och det beror på att antalet födda i Sverige minskar. Minskningen är dock minimal i förhållande till gruppen storlek. På lång sikt är gruppen nästan oförändrad i storlek.År 2070 beräknas den vara 110 000 eller två procent fler än idag och gruppen växer långsammare än hela befolkningen. I diagrammet syns hur förhållandet mellan män och kvinnor förändrats under åren.År 1970 var det för Sverigefödda 103 män per 100 kvinnor i 1 År 2022 motsvarar det en årsinkomst över 213 000 kronor eller 17 750 kronor per månad'},
 {'page_number': 17,
  'sentence_chunk': '![16_image_0.png](16_image_0.png) Olika grupper har olika benägenhet att utvandra. De som kommer till Sverige som flyktingar eller anhöriga stannar kvar i större utsträckning, medan de som kommer för att arbeta eller studera i större utsträckning lämnar landet. Bortsett från år 2023 har utvandringen under senare år varit förhållandevis låg 

In [ ]:
import textwrap

print(f"Query: '{query}'\n")
print("Results:")

for i, score in enumerate(top_results_dot_product[0]):
  print(f"Score: {score:.4f}")
  print("Text:")
  print(textwrap.fill(top_chunks_dot_product[i]["sentence_chunk"]))
  print(f"Page number: {top_chunks_dot_product[i]['page_number']}")
  print("\n")

Query: 'Vilka åldersgrupper kommer att vara mest påverkade av förändringar i dödligheten?'

Results:
Score: 0.7185
Text:
De kommande tio åren förväntas gruppen att minska något, och det beror
på att antalet födda i Sverige minskar. Minskningen är dock minimal i
förhållande till gruppen storlek. På lång sikt är gruppen nästan
oförändrad i storlek.År 2070 beräknas den vara 110 000 eller två
procent fler än idag och gruppen växer långsammare än hela
befolkningen. I diagrammet syns hur förhållandet mellan män och
kvinnor förändrats under åren.År 1970 var det för Sverigefödda 103 män
per 100 kvinnor i 1 År 2022 motsvarar det en årsinkomst över 213 000
kronor eller 17 750 kronor per månad
Page number: 29


Score: 0.6966
Text:
![16_image_0.png](16_image_0.png) Olika grupper har olika benägenhet
att utvandra. De som kommer till Sverige som flyktingar eller anhöriga
stannar kvar i större utsträckning, medan de som kommer för att arbeta
eller studera i större utsträckning lämnar landet. Bortsett

In [64]:
import fitz
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

pdf_path = docs+files[3]
doc = fitz.open(pdf_path)

# Load only the pages in top_chunks_dot_product
pages = [doc.load_page(chunk['page_number']) for chunk in top_chunks_dot_product]

# Convert all pages to images
imgs = [page.get_pixmap(dpi=600) for page in pages]

# Convert all images to numpy arrays
img_arrays = [np.frombuffer(img.samples_mv, dtype=np.uint8).reshape((img.h, img.w, img.n)) for img in imgs]

doc.close()

# Function to display an image
def display_image(i):
    plt.figure(figsize=(8.27, 11.69))
    plt.imshow(img_arrays[i])
    plt.axis('off')
    plt.show()

# Interactive widget to navigate through images
interact(display_image, i=(0, len(img_arrays)-1))